In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo

  Activating project at `d:\CMU\2024_Spring_Course\OCRL\ocrl-piano\conan_tests`
  No Changes to `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\conan_tests\Project.toml`
  No Changes to `D:\CMU\2024_Spring_Course\OCRL\ocrl-piano\conan_tests\Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [4]:
# import piano model
model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

MuJoCo Data object

In [5]:
println("number of geoms: ",model.ngeom)
finger_indices = [115,116,123,124,131,132,141,142,150,151]
step!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        115
        116
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        123
        124
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        131
        132
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        141
        142
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        150
        151
""";

number of geoms: 152
Simulation timestep: 0.005
Positions of joints: [-8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -0.0004997467505765748; -8.82359534359623e-5; -8.82359534359623e-5; -0.000499746750576

## Test DIRCOL: moving index finger left right

In [6]:
tmax = 400
nx = model.nq + model.nv + model.na # State vector dimension
states = zeros(nx, tmax)
for t in 1:tmax
    states[:,t] = get_physics_state(model, data)
    step!(model, data)
end

In [7]:
function wrapped_mj_step(model, data, x, uk)
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] = uk[:]
        # println("hi33")
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # converted_uk = Vector{Float64}()
        # for i in 1:length(uk)
        #     push!(converted_uk, uk[i].value)
        # end
        # converted_x = Vector{Float64}()
        # for i in 1:length(x)
        #     push!(converted_x, x[i].value)
        # end
        # set control 
        data.ctrl[:] = converted_uk[:]
        # println("hi33")
        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # println("hi44")
    # take discrete dynamics step 
    step!(model, data) 
    # println("hi55")
    # return updated state k + 1
    xkp1 = zeros(model.nq + model.nv) 
    xkp1 .= get_physics_state(model, data)
    return xkp1 
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    # stage cost
    cost = 0.0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        cost += 0.5*(xi-xg)'*Q*(xi-xg)+0.5*ui'*R*ui
    end

    # terminal cost 
    xt = Z[idx.x[N]]
    cost += 0.5*(xt-xg)'*Qf*(xt-xg)
    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = params.data
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        # println("xi: ", xi[1:5])
        # print("ui: ", typeof(ui))
        
        c[idx.c[i]] = wrapped_mj_step(model, data, xi, ui) - xip1
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    data = params.data

    # initial_pose_index = wrapped_mj_step(model, data, Z[idx.x[1]], 0)
    # terminal_pose_index = wrapped_mj_step(model, data, Z[idx.x[N]], 0)

    # con_1 =  initial_pose_index - xic
    # con_2 =  terminal_pose_index - xg
    con_1 = Z[idx.x[1]] - xic
    con_2 = Z[idx.x[N]] - xg

    return [con_1; con_2; robohand_dynamics_constraints(params,Z)]
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    return zeros(eltype(Z), 0)
    # return inequality_constraints
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [8]:
function con(params, x)
    [
        robohand_equality_constraints(params, x);
        robohand_inequality_constraints(params, x)
    ]
end

model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)

# initiate time and time steps
dt = 0.1
tf = 1.0
t_vec = 0:dt:tf
N = length(t_vec)

# LQR cost
# Q size only one because we track one finger
Q = diagm(ones(model.nq + model.nv))
R = 0.1*diagm(ones(model.nu))
Qf = 10*diagm(ones(model.nq + model.nv))

# indexing 
idx = create_idx(model.nq + model.nv, model.nu, N)
start_data = [data.qpos[:]; data.qvel[:]]
# calculate final state
start_pos = data.qpos
end_pos = start_pos
end_pos[1] = 0.2
end_pos[2] = 0.25
end_pos[8] = 1.0
end_pos[9] = 1.0
end_pos[10] = 1.0
end_pos[12] = 1.0
end_pos[13] = 1.0
end_pos[14] = 1.0
data.qpos[:] = end_pos[:]
step!(model, data)
end_data = [data.qpos[:]; data.qvel[:]]
reset!(model, data)
# initial and goal state
xic = start_data
xg = end_data

params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)
z0 = 0.001*randn(idx.nz)
println(size(z0))
# ForwardDiff.gradient(_x -> robohand_cost(params, _x), z0)
ForwardDiff.jacobian(_x -> robohand_dynamics_constraints(params, _x), z0)

(2728,)


2280×2728 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.

In [9]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand/scene.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.05
    tf = 1.0
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size only one because we track one finger
    Q = diagm(ones(model.nq + model.nv))
    R = 0.1*diagm(ones(model.nu))
    Qf = 10*diagm(ones(model.nq + model.nv))

    # indexing 
    idx = create_idx(model.nq + model.nv, model.nu, N)
    start_data = [data.qpos[:]; data.qvel[:]]
    # calculate final state
    start_pos = data.qpos
    end_pos = start_pos
    end_pos[1] = 0.2
    end_pos[2] = 0.25
    end_pos[8] = 1.0
    end_pos[9] = 1.0
    end_pos[10] = 1.0
    end_pos[12] = 1.0
    end_pos[13] = 1.0
    end_pos[14] = 1.0
    data.qpos[:] = end_pos[:]
    step!(model, data)
    end_data = [data.qpos[:]; data.qvel[:]]
    reset!(model, data)
    # initial and goal state
    xic = start_data
    xg = end_data
    
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)

    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = zeros(0)
    c_u = zeros(0)
    # initial guess 
    z0 = 0.001*randn(idx.nz)

    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
    # diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [10]:
X, U, t_vec, params = solve_finger_moving(verbose=true)

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...: 26223648
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian..

([[-0.0007846745197280455, -0.0003917509089297878, -0.0019533502632909493, -0.0009873418191516216, 0.00024796619750245473, 0.0023508651568933604, 8.233600734583239e-5, -0.0017956203419610588, 0.0018750587341367791, -0.002497417413160383  …  -0.00034630728025633525, -0.0013495948682729767, 0.0008964087296992461, 0.0019567961440965422, 0.0004001459538073, 0.0008056440938755437, 0.00020720831377491362, -0.0005681527212732391, -0.0006879205461642219, -0.00022321142003899167], [-0.0002649876711432141, 0.0007803803574051869, -0.0003461498348362619, 4.5506089180187094e-5, 0.00013647152845238297, 0.0001241263945292875, 0.0010337970484934718, -0.000562818539987372, -0.0003378401410868982, -0.001240898921181623  …  0.0011548990405474038, -0.0010128452629563694, -0.0002428639568870012, 0.0011958437230158733, 0.0016904799042422789, -0.0023860917308387796, -0.0002498833659374493, 0.0006831030989025012, 0.0003966837370340022, 0.0012340859014446863], [-0.00048021004059094, -0.0014586161223070781, -0.